## Tablas generadas en la capa Gold

Este notebook ha generado las siguientes tablas en la capa **Gold**:

- **gold_users** → Dimensión de usuarios.
- **gold_tags** → Dimensión de tags.
- **gold_calendar** → Dimensión de fechas.
- **gold_fact_posts** → Tabla de hechos con publicaciones.
- **gold_edges_users** → Relación de usuarios con sus seguidores.
- **gold_edges_user_tag** → Relación de usuarios con tags en sus posts.
- **gold_edges_tags** → Relación de tags que aparecen juntos.

### ¿Cómo usar estas tablas en Power BI?

- **Para grafo de usuarios:** Usar `gold_users` y `gold_edges_users` con `user_id` y `follower_id`.
- **Para grafo de tags:** Usar `gold_tags` y `gold_edges_tags` con `tag_source` y `tag_target`.
- **Para analizar menciones en posts:** Usar `gold_edges_user_tag`.



In [1]:
%run NB_Credentials

StatementMeta(, bc229c50-06d7-44f6-bb0d-75efd710b97d, 3, Finished, Available, Finished)

In [2]:
import pyspark.sql.functions as F
from pyspark.sql.functions import col, explode, split, lower, trim
import time

StatementMeta(, bc229c50-06d7-44f6-bb0d-75efd710b97d, 4, Finished, Available, Finished)

In [3]:
# Iniciar tiempo de ejecución
start_time = time.time()

StatementMeta(, bc229c50-06d7-44f6-bb0d-75efd710b97d, 5, Finished, Available, Finished)

In [4]:
# Nombres de las tablas
SILVER_FACT_POST = "silver_fact_post"
SILVER_DIM_USERS = "silver_dim_users"
SILVER_DIM_TAGS = "silver_dim_tags"
SILVER_DIM_DATE = "dim_date"

# Definir nombres de las tablas en Gold
GOLD_USERS = "gold_users"
GOLD_TAGS = "gold_tags"
GOLD_CALENDAR = "gold_calendar"
GOLD_FACT_POSTS = "gold_fact_posts"
GOLD_EDGES_USERS = "gold_edges_users"
GOLD_EDGES_USER_TAG = "gold_edges_user_tag"
GOLD_EDGES_TAGS = "gold_edges_tags"

StatementMeta(, bc229c50-06d7-44f6-bb0d-75efd710b97d, 6, Finished, Available, Finished)

In [5]:
#  Cargar tablas de Silver
df_fact_post = spark.read.format("delta").load(f"{SILVER_PATH}/{SILVER_FACT_POST}")
df_dim_users = spark.read.format("delta").load(f"{SILVER_PATH}/{SILVER_DIM_USERS}")
df_dim_tags = spark.read.format("delta").load(f"{SILVER_PATH}/{SILVER_DIM_TAGS}")
df_date = spark.read.format("delta").load(f"{SILVER_PATH}/{SILVER_DIM_DATE}")

StatementMeta(, bc229c50-06d7-44f6-bb0d-75efd710b97d, 7, Finished, Available, Finished)

In [6]:
# Generación de tabla POSTS en gold
df_fact_posts_gold = df_fact_post.select(
    F.col("author").alias("user_id"),
    F.col("date").alias("post_date"),
    F.col("post").alias("post_text")
)

# Guardar en Gold
df_fact_posts_gold.write \
    .format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .save(f"{GOLD_PATH}/{GOLD_FACT_POSTS}")

StatementMeta(, bc229c50-06d7-44f6-bb0d-75efd710b97d, 8, Finished, Available, Finished)

In [7]:
# Generación de tabla USERS en gold
df_users_gold = df_dim_users.select(
    F.col("author").alias("user_id"),
    "display_name",
    "description",
    "avatar",
    "followers_count",
    "follows_count",
    "posts_count",
    "followers_list" 
)

# Guardar en Gold
df_users_gold.write \
    .format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .save(f"{GOLD_PATH}/{GOLD_USERS}")

StatementMeta(, bc229c50-06d7-44f6-bb0d-75efd710b97d, 9, Finished, Available, Finished)

In [8]:
# Generación de tabla USERS_duplicate en gold
df_users_gold = df_dim_users.select(
    F.col("author").alias("user_id"),
    "display_name",
    "description",
    "avatar",
    "followers_count",
    "follows_count",
    "posts_count",
    "followers_list" 
)

# Guardar en Gold
df_users_gold.write \
    .format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .save(f"{GOLD_PATH}/{GOLD_USERS}_dup")

StatementMeta(, bc229c50-06d7-44f6-bb0d-75efd710b97d, 10, Finished, Available, Finished)

In [9]:
# Generación de tabla TAGS en gold
df_tags_gold = df_dim_tags.select(
    F.col("hashtag").alias("hashtag_id"),
    "author",               
    "date",                  
    "related_hashtags"       
).distinct()  

# Guardar en Gold
df_tags_gold.write \
    .format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .save(f"{GOLD_PATH}/{GOLD_TAGS}")

StatementMeta(, bc229c50-06d7-44f6-bb0d-75efd710b97d, 11, Finished, Available, Finished)

In [10]:
# Generación de tabla TAGS_duplicate en gold
df_tags_gold = df_dim_tags.select(
    F.col("hashtag").alias("hashtag_id"),
    "author",               
    "date",                  
    "related_hashtags"       
).distinct()  

# Guardar en Gold
df_tags_gold.write \
    .format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .save(f"{GOLD_PATH}/{GOLD_TAGS}_dup")

StatementMeta(, bc229c50-06d7-44f6-bb0d-75efd710b97d, 12, Finished, Available, Finished)

In [11]:
# Generación de tabla CALENDAR en gold
df_date.write.format("delta").mode("overwrite").option("mergeSchema", "true").save(f"{GOLD_PATH}/{GOLD_CALENDAR}")

StatementMeta(, bc229c50-06d7-44f6-bb0d-75efd710b97d, 13, Finished, Available, Finished)

## Generación de Relaciones para el Análisis de Grafos

In [12]:
# Tabla gold_edges_users → Relaciones entre usuarios (basadas en seguidores)
df_edges_users = df_dim_users.select(
    F.col("author").alias("user_id"),
    F.explode(F.split(F.col("followers_list"), ", ")).alias("follower_id")
).filter(F.col("follower_id") != "")

# Guardar en Gold
df_edges_users.write \
    .format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .save(f"{GOLD_PATH}/{GOLD_EDGES_USERS}")
 
 # display(df_edges_users)

StatementMeta(, bc229c50-06d7-44f6-bb0d-75efd710b97d, 14, Finished, Available, Finished)

In [13]:
# Tabla gold_edges_user_tag → Relaciones entre usuarios y tags con peso
df_edges_user_tag = df_fact_post.alias("p") \
    .join(
        df_dim_tags.alias("t"),
        (F.col("p.author") == F.col("t.author")) & (F.col("p.date") == F.col("t.date")),
        "inner"
    ) \
    .groupBy(
        F.col("p.author").alias("user_id"),
        F.col("t.hashtag").alias("hashtag_id")
    ) \
    .agg(F.count("*").alias("weight"))  # Número de veces que el usuario usa el tag

# Guardar en Gold 
df_edges_user_tag.write \
    .format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .save(f"{GOLD_PATH}/{GOLD_EDGES_USER_TAG}")

# Mostrar resultados
# display(df_edges_user_tag)

StatementMeta(, bc229c50-06d7-44f6-bb0d-75efd710b97d, 15, Finished, Available, Finished)

In [14]:
# Tabla gold_edges_tags → Relaciones entre tags por co-ocurrencia en la misma fecha
df_edges_tags = df_dim_tags.alias("a") \
    .join(
        df_dim_tags.alias("b"),
        (F.col("a.date") == F.col("b.date")) & (F.col("a.hashtag") < F.col("b.hashtag")),
        "inner"
    ) \
    .groupBy(
        F.col("a.hashtag").alias("tag_source"),
        F.col("b.hashtag").alias("tag_target")
    ) \
    .agg(F.count("*").alias("weight"))  # Número de veces que aparecen juntos

# Guardar en Gold 
df_edges_tags.write \
    .format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .save(f"{GOLD_PATH}/{GOLD_EDGES_TAGS}")

# display(df_edges_tags)

StatementMeta(, bc229c50-06d7-44f6-bb0d-75efd710b97d, 16, Finished, Available, Finished)

In [15]:
# Calcular tiempo de ejecución
end_time = time.time()
execution_time = end_time - start_time
print(f"✅ Transformación completada en {execution_time:.2f} segundos.")

StatementMeta(, bc229c50-06d7-44f6-bb0d-75efd710b97d, 17, Finished, Available, Finished)

✅ Transformación completada en 69.03 segundos.
